In [ ]:
import openreview
import iclr19 as ICLR

client = openreview.Client()
print(client.baseurl)

## Setup dev instance

In [ ]:
# create dummy users:
# - author and co-author
# - reviewer user 
# - areachair user 
# - pc user 
# - author and reviewer user 
# - author and area chair user 

pw = '1234'
user_credentials = [
    {'email': 'author@iclr19test.net', 'first': 'Author', 'last': 'ICLR', 'password': pw},
    {'email': 'reviewer@iclr19test.net', 'first': 'Reviewer', 'last': 'ICLR', 'password': pw},
    {'email': 'areachair@iclr19test.net', 'first': 'AreaChair', 'last': 'ICLR', 'password': pw},
    {'email': 'programchair@iclr19test.net', 'first': 'ProgramChair', 'last': 'ICLR', 'password': pw},
    {'email': 'authorreviewer@iclr19test.net', 'first': 'AuthorReviewer', 'last': 'ICLR', 'password': pw},
    {'email': 'authorareachair@iclr19test.net', 'first': 'AuthorAreaChair', 'last': 'ICLR', 'password': pw}
]

for c in user_credentials:
    response = client.register_user(**c)

In [ ]:
client.register_user(**{'email': 'newauthor@iclr19test.net', 'first': 'NewAuthor', 'last': 'ICLR', 'password': '1234'})

In [ ]:
ICLR.program_chairs.members=['programchair@iclr19test.net']
client.post_group(ICLR.program_chairs)

# post the groups in the conference's path
iclr_groups = openreview.tools.build_groups(ICLR.conference.id)
for group in iclr_groups:
    try:
        existing_group = client.get_group(group.id)
    except openreview.OpenReviewException as e:
        posted_group = client.post_group(group)
        print(posted_group.id)

# post the conference group with updated parameters
client.post_group(ICLR.conference)

## Webfield download/upload

In [ ]:
# # Download ICLR homepage
# conf = client.get_group(ICLR.CONFERENCE_ID)
# with open('../webfield/iclr-homepage.js','w') as f:
#     f.write(conf.web)

In [ ]:
# # Download bid webfield
# bid = client.get_invitation('ICLR.cc/2019/Conference/-/Add_Bid')
# with open('../webfield/bidWebfield.js','w') as f:
#     f.write(bid.web)

In [ ]:
# # Download area chair webfield
# with open('../webfield/areachairWebfield.js','w') as f:
#     f.write(client.get_group(ICLR.AREA_CHAIRS_ID).web)

In [ ]:
# Upload bid webfield
bid = client.get_invitation('ICLR.cc/2019/Conference/-/Add_Bid')
with open('../webfield/bidWebfield.js') as f:
    bid.web = f.read()
    client.post_invitation(bid)

In [ ]:
# Upload ICLR homepage
conf = client.get_group(ICLR.CONFERENCE_ID)
with open('../webfield/homepage.js') as f:
    conf.web = f.read()
    client.post_group(conf)

In [ ]:
# Upload reviewer webfield
with open('../webfield/reviewerWebfield.js','r') as f:
    reviewers = client.get_group('ICLR.cc/2019/Conference/Reviewers')
    reviewers.web = f.read()
    client.post_group(reviewers)

In [ ]:
# Upload areachair webfield
with open('../webfield/areachairWebfield.js','r') as f:
    areachairs = client.get_group('ICLR.cc/2019/Conference/Area_Chairs')
    areachairs.web = f.read()
    client.post_group(areachairs)

In [ ]:
# Upload author webfield
with open('../webfield/authorWebfield.js','r') as f:
    authors = client.get_group('ICLR.cc/2019/Conference/Authors')
    authors.web = f.read()
    client.post_group(authors)

In [ ]:
# Upload program chair webfield
with open('../webfield/programchairWebfield.js','r') as f:
    program_chairs = client.get_group('ICLR.cc/2019/Conference/Program_Chairs')
    program_chairs.web = f.read()
    client.post_group(program_chairs)

# Demo Begins Here

### Set up reviewer recruitment and registration

In [ ]:
# post reviewer groups that we can use to track those that were invited, 
# and those that accepted or declined the invitation.

ICLR.reviewers.members = [
    'melisabok@gmail.com',
    'reviewer@iclr19test.net',
    'authorreviewer@iclr19test.net'
]

client.post_group(ICLR.reviewers)
client.post_group(ICLR.reviewers_invited)
client.post_group(ICLR.reviewers_declined)

In [ ]:
client.get_notes(invitation='sdfa')

In [ ]:
# get existing roots
existing_roots = client.get_notes(invitation=ICLR.registration_root_invitation.id)
print(existing_roots)

In [ ]:
# post root registration invitation and note
client.post_invitation(ICLR.registration_root_invitation)
if existing_roots:
    registration_root_note = existing_roots[0]
else:    
    registration_root_note = client.post_note(ICLR.registration_root_note)

In [ ]:
registration_root_note.id

In [ ]:
client.post_invitation(openreview.invitations.from_template(ICLR.data_consent_template, registration_root_note))
client.post_invitation(openreview.invitations.from_template(ICLR.subject_areas_template, registration_root_note))
client.post_invitation(openreview.invitations.from_template(ICLR.confirm_profile_template, registration_root_note))

In [ ]:
# do the same for area chairs.
#ICLR.area_chairs.members = ['ICLRareachair@openreview.net']

ICLR.area_chairs.members = [
    'areachair@iclr19test.net',
    'authorareachair@iclr19test.net'
]

client.post_group(ICLR.area_chairs)
client.post_group(ICLR.area_chairs_invited)
client.post_group(ICLR.area_chairs_declined)

In [ ]:
# update the Authors group with the per-paper author groups

member_set = set()
authorgroups = client.get_groups('/'.join([ICLR.CONFERENCE_ID, 'Paper.*', 'Authors']))
for g in authorgroups:
    member_set.update([g.id])
    
print(member_set)

ICLR.authors.members = list(member_set)
client.post_group(ICLR.authors)

In [ ]:
authors = client.get_group(ICLR.AUTHORS_ID)
with open('../webfield/authorWebfield.js') as f:
    authors.web = f.read()
    client.post_group(authors)

In [ ]:
# post the recruitment invitations
client.post_invitation(ICLR.recruit_reviewers)

In [ ]:
# define some potential reviewers and area chairs
rev_email = 'spector@cs.umass.edu'
rev_firstname = 'ICLR Reviewer'

In [ ]:
# deploy reviewer invitation
openreview.tools.recruit_reviewer(client, rev_email, rev_firstname,
                                 hash_seed = ICLR.HASH_SEED,
                                 recruit_reviewers_id = ICLR.RECRUIT_REVIEWERS_ID,
                                 recruit_message = ICLR.RECRUIT_REVIEWERS_MESSAGE,
                                 recruit_message_subj = ICLR.RECRUIT_MESSAGE_SUBJ,
                                 reviewers_invited_id = ICLR.REVIEWERS_INVITED_ID
                                 )

### Double-blind Paper Submission

In [ ]:
# post the submission and blind submission invitations
posted_submission_inv = client.post_invitation(ICLR.submission_inv)
posted_blind_inv = client.post_invitation(ICLR.blind_submission_inv)

The following section is for the ICLR demo.

It collects papers from ICLR 2018 and posts them as if they were ICLR 2019 submissions.

In [ ]:
# collect old ICLR papers and post them
old_iclr_papers = client.get_notes(invitation='ICLR.cc/2018/Conference/-/Submission', limit=5)
print(len(old_iclr_papers))

In [ ]:
old_iclr_papers[0].writers

In [ ]:
# post the ICLR papers
for paper in old_iclr_papers:
    if 'ICLR.cc/2018/Conference' in paper.writers:
        paper.writers.remove('ICLR.cc/2018/Conference')
        paper.writers.append('ICLR.cc/2019/Conference')
    if 'TL;DR' in paper.content:
        paper.content.pop('TL;DR')
    
    posted_submission = client.post_note(openreview.Note.from_json({
        'writers': paper.writers,
        'readers': [ICLR.CONFERENCE_ID] + paper.content['authorids'] + paper.signatures,
        'content': paper.content,
        'invitation': ICLR.SUBMISSION_ID
    }))
    print(posted_submission.content['title'])
    

### Area Chair and Reviewer bidding

In [ ]:
# OpenReview lets area chairs and reviewers (and whoever else) bid on papers.
ICLR.add_bid.invitees = [ICLR.REVIEWERS_ID, ICLR.AREA_CHAIRS_ID]
client.post_invitation(ICLR.add_bid)
# client.post_invitation(ICLR.add_wager)

### Reviewer recommendations from area chairs

In [ ]:
# get all currently submitted papers
ICLR_submissions = openreview.tools.get_all_notes(client, ICLR.BLIND_SUBMISSION_ID)

# Add reviewer rec tag invitation to the last five papers
for paper in ICLR_submissions[0:5]:
    rec_inv = openreview.invitations.from_template(ICLR.ac_recommendation_template, paper)
    rec_inv.invitees = ['ICLRareachair@openreview.net']
    posted_inv = client.post_invitation(rec_inv)
    print posted_inv.id

In [ ]:
# convert reviewers to canonical identities

reviewers_group = client.get_group(ICLR.REVIEWERS_ID)
for member in reviewers_group.members:
    if '@' in member:
        profile = client.get_profile(member)
        client.remove_members_from_group(reviewers_group, member)
        client.add_members_to_group(reviewers_group, profile.id)
        

In [ ]:
ICLR.area_chairs.members

### Official Reviews, Official Comments, and Open Comments

In [ ]:
# get all currently submitted papers
ICLR_submissions = openreview.tools.iterget_notes(client, invitation=ICLR.BLIND_SUBMISSION_ID)
area_chairs = client.get_group(id=ICLR.AREA_CHAIRS_ID)
reviewers = client.get_group(id=ICLR.REVIEWERS_ID)
# assign some papers

for paper in ICLR_submissions:
    for reviewer in reviewers.members:
        openreview.tools.assign(client, paper.number, ICLR.CONFERENCE_ID,
            reviewer_to_add = reviewer,
            parent_label = 'Reviewers',
            individual_label = 'AnonReviewer',
            individual_group_params = {
                'readers': [
                    ICLR.AREA_CHAIRS_ID, 
                    ICLR.PROGRAM_CHAIRS_ID
                ]
            })
    for areachair in area_chairs.members:
        openreview.tools.assign(client, paper.number, ICLR.CONFERENCE_ID,
            reviewer_to_add = areachair,
            parent_label = 'Area_Chairs',
            individual_label = 'Area_Chair',
            individual_group_params = {
                'readers': [
                    ICLR.PROGRAM_CHAIRS_ID
                ]
            })

In [ ]:
anonreviewers = client.get_groups('ICLR.cc/2019/Conference/Paper.*/AnonReviewer.*')
acs = client.get_groups('CLR.cc/2019/Conference/Paper.*/Area_Chair.*')

for g in anonreviewers + acs:
    g.readers += ['ICLR.cc/2019/Conference/Program_Chairs']
    client.post_group(g)

In [ ]:
ICLR_submissions = openreview.tools.iterget_notes(client, invitation=ICLR.BLIND_SUBMISSION_ID)
for paper in ICLR_submissions:
    reviewer_group = client.get_group('/'.join([ICLR.CONFERENCE_ID, 'Paper{}'.format(paper.number), 'Reviewers']))
    reviewer_group.readers = [
        "ICLR.cc/2019/Conference",
        "ICLR.cc/2019/Conference/Program_Chairs"
    ]
    client.post_group(reviewer_group)

In [ ]:
# get all currently submitted papers
ICLR_submissions = openreview.tools.iterget_notes(client, invitation=ICLR.BLIND_SUBMISSION_ID)
ICLR_originals = openreview.tools.iterget_notes(client, invitation=ICLR.SUBMISSION_ID)
# enable reviews
for paper in ICLR_submissions:
#     client.post_invitation(openreview.invitations.from_template(ICLR.official_review_template, paper))
#     client.post_invitation(openreview.invitations.from_template(ICLR.official_comment_template, paper))
#     client.post_invitation(openreview.invitations.from_template(ICLR.meta_review_template, paper))
    client.post_invitation(openreview.invitations.from_template(ICLR.review_rating_template, paper))

# for paper in ICLR_originals:
#     client.post_invitation(openreview.invitations.from_template(ICLR.add_revision_template, paper))

In [ ]:
# get all currently submitted papers
ICLR_submissions = openreview.tools.get_all_notes(client, ICLR.BLIND_SUBMISSION_ID)

for paper in ICLR_submissions:
    metadata_params = {
        'forum': paper.id,
        'replyto': paper.id,
        'invitation': ICLR.REVIEWER_METADATA_ID,
        'readers': {
            'values': [
                ICLR.CONFERENCE_ID
            ]
        },
        'writers': {
            'values': [ICLR.CONFERENCE_ID]
        },
        'signatures': {
            'values': [ICLR.CONFERENCE_ID]},
        'content': {
            'groups': {
                ICLR.REVIEWERS_ID: [
                    {
                        'scores': {'affinity': 1.0},
                        'constraints': {'cmtConflict': '-inf'},
                        'userId': '~Red_Reviewer1'
                    },
                    {
                        'scores': {'affinity': 1.0},
                        'constraints': {'cmtConflict': '-inf'},
                        'userId': '~Blue_Reviewer1'
                    },
                    {
                        'scores': {'affinity': 1.0},
                        'constraints': {'cmtConflict': '-inf'},
                        'userId': '~Green_Reviewer1'
                    },
                    {
                        'scores': {'affinity': 1.0},
                        'constraints': {'cmtConflict': '-inf'},
                        'userId': '~ICLR_Reviewer1'
                    }
                ]
            }
        }
    }

In [ ]:
import random

client.post_invitation(ICLR.assignment_inv)

for paper in ICLR_submissions:
    finalscore = random.random()
    assignment_params = {
        'invitation': ICLR.ASSIGNMENT_INV_ID,
        'readers': [ICLR.CONFERENCE_ID],
        'writers': [ICLR.CONFERENCE_ID],
        'signatures': [ICLR.CONFERENCE_ID],
        'forum': paper.id,
        'replyto': paper.id,
        'content': {
            'label': 'reviewers',
            'assignedGroups': [
                {
                    "finalScore": 1.0,
                    "scores": {
                        "affinity": 1.0
                    },
                    "constraints": {},
                    "userId": "~Red_Reviewer1"
                },
            ],
            'alternateGroups': [
                {
                    "finalScore": random.random(),
                    "scores": {
                        "affinity": random.random()
                    },
                    "constraints": {'conflict': '-inf'},
                    "userId": "~Blue_Reviewer1"
                },
                {
                    "finalScore": random.random(),
                    "scores": {
                        "affinity": random.random()
                    },
                    "constraints": {},
                    "userId": "~Green_Reviewer1"
                },
                {
                    "finalScore": random.random(),
                    "scores": {
                        "affinity": random.random()
                    },
                    "constraints": {},
                    "userId": "~ICLR_Reviewer1"
                },
            ]
        }   
    }
    
    client.post_note(openreview.Note(**assignment_params))


In [ ]:
client.post_invitation(ICLR.config_inv)
config_note = client.post_note(openreview.Note(**{
    'invitation': ICLR.CONFIG_INV_ID,
    'readers': [ICLR.CONFERENCE_ID],
    'writers': [ICLR.CONFERENCE_ID],
    'signatures': [ICLR.CONFERENCE_ID],
    'content': {
        'label': 'reviewers',
        'configuration': {
            'minusers': 1,
            'maxusers': 1,
            'minpapers': 0,
            'maxpapers': 8,
            'alternates': 5,
            'weights': {
                'affinity': 1.0,
                'acRecommendation': 3.0,
                'bidScore': 1.0
            }
        },
        'constraints': {},
        'paper_invitation': ICLR.BLIND_SUBMISSION_ID,
        'metadata_invitation': ICLR.REVIEWER_METADATA_ID,
        'assignment_invitation': ICLR.ASSIGNMENT_INV_ID,
        'match_group': ICLR.REVIEWERS_ID,
        'status': 'complete' # 'queued', 'processing', 'complete' or 'error'
    }
}))